In [1]:
import pandas as pd
import cmbagent
import os
from extract_parquet import extract_file_path
import numpy as np
import matplotlib.pyplot as plt
from ins import run_single_function
import pickle as pl

In [2]:
df = pd.read_csv('data/camb_data_v3.csv')


In [3]:
stats = {0: [1, 1, 0, 0, 0, 0, 1, 0, 0, 1],
 1: [1, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 2: [0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
 3: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 6: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 7: [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
 8: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 10: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 11: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 12: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],}

In [ ]:
trials = 10
agent = 'camb_context'

for i in range(len(df)):
    # Skip completed prompts
    if i in stats and len(stats[i]) == trials:
        print(f"Prompt {i} already completed, skipping.")
        continue
    
    # Initialize if not present
    if i not in stats:
        stats[i] = []

    for j in range(len(stats[i]), trials):  # resume unfinished trials
        prompt = df.prompt.iloc[i]
        reference_code = df.reference_code.iloc[i]
        work_dir = f"/Users/antoidicherianlonappan/Workspace/cmbagent_output/evals/prompt_{agent}_{i}/trial_{j}/"
        os.makedirs(work_dir, exist_ok=True)

        results = cmbagent.one_shot(
            prompt,
            max_rounds=50,
            agent=agent,
            work_dir=work_dir,
        )

        resultfile = os.path.join(results['final_context']['work_dir'],
                                  results['final_context']['database_path'],
                                  'result.csv')
        if not os.path.isfile(resultfile):
            stats[i].append(0)
            continue
        else:
            l_llm, cl_llm = np.loadtxt(resultfile, delimiter=',', skiprows=1).T

        l_true, cl_true = run_single_function(reference_code)
        if len(l_llm) != len(l_true):
            stats[i].append(0)
            continue

        plt.loglog(l_llm, cl_llm, label='LLM')
        plt.loglog(l_true, cl_true, label='True')
        plt.legend()
        plt.savefig(os.path.join(work_dir, 'cl_plot.png'))
        plt.close()

        if np.allclose(l_llm, l_true) and np.allclose(cl_llm, cl_true, rtol=1e-1):
            stats[i].append(1)
        else:
            stats[i].append(0)
    
    print(f"Prompt {i} done, stats: {stats[i]}")

Prompt 0 already completed, skipping.
Prompt 1 already completed, skipping.
Prompt 2 already completed, skipping.
Prompt 3 already completed, skipping.
Prompt 4 already completed, skipping.
Prompt 5 already completed, skipping.
Prompt 6 already completed, skipping.
Prompt 7 already completed, skipping.
Prompt 8 already completed, skipping.
Prompt 9 already completed, skipping.
Prompt 10 already completed, skipping.
Prompt 11 already completed, skipping.
Prompt 12 already completed, skipping.
_User (to chat_manager):

Calculate the delensing efficiency of the Cosmic Microwave Background (CMB) B-mode polarization power spectrum for a flat Lambda CDM cosmology using the following parameters with CAMB given the lensing noise power:
Hubble constant ($H_0$): 67.5 km/s/Mpc
Baryon density ($\Omega_b h^2$): 0.022
Cold dark matter density ($\Omega_c h^2$): 0.122
Neutrino mass sum ($\Sigma m_
u$): 0.06 eV
Curvature ($\Omega_k$): 0
Optical depth to reionization ($	au$): 0.06
Scalar amplitude ($A_s

In [5]:

pl.dump(stats, open(f"data/camb_data_v3_{agent}.pkl", 'wb'))

In [30]:
codefile = os.path.join(results['final_context']['work_dir'],extract_file_path(results['final_context']['current_codebase']))